## __Code now works.__<br>
<br>

__To Do:__<br>

- Adding code that allows for bond angles (if needed). <br>
- General tweaking and cleaning up of the code. <br>
- Add more dictionaries as needed. <br>



In [2]:
import hoomd,imp
import hoomd
from hoomd import *
from hoomd import md
import numpy as np
import gsd.hoomd
import time 
from inspect import currentframe

class SimBox:
    
    ###Creates Initial Snapshot
    def MakeSnapshot(self, boxsize, nparticles): 
        
        snapshot=hoomd.data.make_snapshot(N=nparticles,box=hoomd.data.boxdim(L=boxsize+nparticles),
                                          bond_types=['tether'],particle_types=self.GenParticleTypes(nparticles, debug=True))
        return snapshot
    
    
    #Only used for initializing the system, do not use outside of MakeSnapshot!
    def GenParticleTypes(self, nparticles, debug=None):
        counter=0
        #List that will contain the particle types. Enable debug to print.
        GPT = [] 
        for k in range(nparticles):
            counter=k
            GPT.append('C%d'%counter)
        if(debug==True):
            print("DEBUG: GenParticleTypes: Particle Types = ",GPT)
        return GPT


In [3]:
from inspect import currentframe

class Polymer:

    def SetPolyProperties(self, nparticles, boxsize, diam=None, debug=None):
        if(diam==None):
            diam=1.0 #default diameter
        for i in range(nparticles):
            system.particles.get(i).position = [-(boxsize-nparticles)+i,0,0]
            system.particles.get(i).diameter = 1.0
            system.particles.get(i).type = ('C%d'%(i))
            if(debug==True):
                print("DEBUG: SetPolyProperties: Particle %d is at "%i, [nparticles+i,0,0])
    
    #MakePolyTage is just a debugging tool, ignore. 
    def MakePolyTags(self, nparticles, debug=None):
        P_Tag_List = []
        for i in range(nparticles):
            system.particles.get(i).type=('C%d'%(i+1))
            P_Tag_List.append('C%d'%(i+1))
        if(debug==True):
            print("DEBUG: SetPolyProperties: P_Tag_List is ", P_Tag_List)
        
    def PatchTypes(self, tot_npatch, debug=None):
        patch_types = []
        for i in range(tot_npatch):
            system.particles.types.add('%d'%(i+1))
            patch_types.append('%d'%(i+1))
        if(debug==True):
            print("DEBUG: PatchTypes: ",patch_types)
        return patch_types

            
    
    def MakeBonds(self, nparticles, debug=None):
        DebugBonds = []
        f=0
        for f in range(nparticles-1):
            system.bonds.add('tether',f,f+1)
            DebugBonds.append([f,f+1])
            f+=1
        if(debug==True):
            print("DEBUG: MakeBonds: Bonds = ", DebugBonds)  

In [4]:
from inspect import currentframe

class Debug:
    def PrintAllParticleTags(self, nparticles, tot_npatch):
        X = []
        for i in range(nparticles+tot_npatch):
            X.append(system.particles[i].tag)
        print("DEBUG: List of All Particle Tags: ", X)
    def PrintSysParticles(self, debug=None):
        if(debug==True):
            for i in range(len(system.particles)):
                print(system.particles[i])
    def PrintLJPairs(self, sig, types):
        for i in range(0,len(sig)):
            for j in range(i,len(sig)):
                print([types[i],types[j]])
    def DebugMessage(self, string, variable, debug=None):
        if(debug==True):
            cf = currentframe()
            print("%s"%string, variable, " at line ", cf.f_back.f_lineno)
    def DebugInfo(self):
        print("Debug Message Format Is As Follows:\n DEBUG: Function: Parameter = Value at line <linenumber>")



In [5]:
from inspect import currentframe

class MomentOfInertia:
  
    #Principle Moment Calculator
    #added an option to return the inertia tensor for debugging
    def Moment(self, coord, Mass, return_tensor = None, debug=None):
        DB = Debug()
        #Referencing local variables
        cx,cy,cz = 0.0,0.0,0.0
        x,y,z = 0.0,0.0,0.0
        cXYZ = []
        Ixx,Ixy,Ixz,Iyy,Iyz,Izz = 0.0,0.0,0.0,0.0,0.0,0.0
        TotalMass = sum(Mass)
        if(debug==True):
            print("DEBUG: Moment: len(coord)= ", len(coord))
            print("DEBUG: Moment: TotalMass = ", TotalMass)
        for i in range(len(Mass)):
                x += Mass[i]*coord[i][0]
                y += Mass[i]*coord[i][1]
                z += Mass[i]*coord[i][2]
                if(debug==True):
                    print("DEBUG: Moment: x = Mass[%d]"%i +"*coord[%d][0]"%i +" =%f "%x)
                    print("DEBUG: Moment: y = Mass[%d]"%i +"*coord[%d][1]"%i +" =%f "%y)
                    print("DEBUG: Moment: z = Mass[%d]"%i +"*coord[%d][2]"%i +" =%f "%z)
        cx = x/TotalMass
        cy = y/TotalMass
        cz = z/TotalMass
        if(debug==True):
            print("DEBUG: Moment: cx = x/TotalMass = ", cx)
            print("DEBUG: Moment: cy = y/TotalMass = ", cy)
            print("DEBUG: Moment: cz = y/TotalMass = ", cz)
        com = [cx,cy,cz]
        for i in range(len(coord)):
            cXYZ.append([coord[i][0]-com[0], coord[i][1]-com[1], coord[i][2]-com[2]]) 
        if(debug==True):
            print("DEBUG: Moment: cXYZ = ", cXYZ)
            print("DEBUG: Moment: len(cXYZ) = ", len(cXYZ))
        #Constructs Inertia Tensor
        #Useful Property: Ixy=Iyx, Ixz=Izx, Iyz=Izy
        for i in range(len(coord)):
            Ixx += Mass[i]*(cXYZ[i][1]**2 + cXYZ[i][2]**2)
            Iyy += Mass[i]*(cXYZ[i][0]**2 + cXYZ[i][2]**2)
            Izz += Mass[i]*(cXYZ[i][0]**2 + cXYZ[i][1]**2)
            Ixy += -Mass[i]*cXYZ[i][0]*cXYZ[i][1]
            Ixz += -Mass[i]*cXYZ[i][0]*cXYZ[i][2]
            Iyz += -Mass[i]*cXYZ[i][1]*cXYZ[i][2]
        if(debug==True):
            print("DEBUG: Moment: Ixx = ", Ixx)
            print("DEBUG: Moment: Iyy = ", Iyy)
            print("DEBUG: Moment: Izz = ", Izz)
            print("DEBUG: Moment: Ixy = ", Ixy)
            print("DEBUG: Moment: Ixz = ", Ixz)
            print("DEBUG: Moment: Iyz = ", Iyz)

        #Computes eignevalues, finds principle moments
        Imatrix = np.matrix([[Ixx,Ixy,Ixz],[Ixy,Iyy,Iyz],[Ixz,Iyz,Izz]])
        Idiag = np.linalg.eig(Imatrix)
        if(debug==True):
            print("DEBUG: Moment: Idiag = ",Idiag)
            print("DEBUG: Moment: Idiag[0] = ",Idiag[0])
        comX = 0-com[0]
        comY = 0-com[1] 
        for i in range(len(coord)):
            coord[i] = [coord[i][0]+comX,coord[i][1]+comY,coord[i][2]]
        if(return_tensor==True):
            return Idiag[0], Imatrix
        else:
            return Idiag[0]

In [7]:
import hoomd,imp
import hoomd
from hoomd import *
from hoomd import md
import numpy as np
import gsd.hoomd
import time 
from inspect import currentframe

#Filename (date and time will be appended onto the file name)
filename = "Patchy_Polymer"
#Box Size
boxsize = 1
#Number of constituent particles
nparticles = 10
#Total number of patches in sim
tot_npatch = 4
#Rigid Diameter
diam = 1.0
#List of Patch Diameters
pdiam = [1.0]
#LJ Cutoff
rcut = 2.5
#Random Seed
SEED = np.random.randint(0,100000000)
#Empty list for setting up sig values
Sigma = []
#Empty list for setting up eps values
Epsilon = []
#Allows me to make unique file names
t = time.strftime(time.strftime("%d %b %H:%M:%S", time.gmtime()))
#Just for flexability 
def Angle(angle):
    angle = angle*np.pi/180
    return angle
##----------SETUP----------##
context.initialize("")


DB = Debug()
DB.DebugInfo()
SB = SimBox()
MoI = MomentOfInertia()
Poly = Polymer()

#Creates snapshot, 
"""also adds initial particle types"""
snapshot=hoomd.data.make_snapshot(N=nparticles,box=hoomd.data.boxdim(Lx=11, Ly=4, Lz=4),
        bond_types=['tether'],particle_types=SB.GenParticleTypes(nparticles, debug=False))

system = hoomd.init.read_snapshot(snapshot)


patch_types = Poly.PatchTypes(tot_npatch, debug=False) ##Adds patch Patch Types from [1,tot_npatch]


Poly.SetPolyProperties(nparticles, boxsize, diam=1.0, debug=False)

types = system.particles.types
DB.DebugMessage("DEBUG: types = ", types, debug=False)

##----------Builds Dictionaries for LJ Parameters---------##
for i in range(len(types)):
    Sigma.append(('%s'%types[i], 1.0))
    Epsilon.append(('%s'%types[i], 0.1)) 
sig = dict(Sigma)
eps = dict(Epsilon)

#Automatically sets sig and eps values for patches 
for i in range(len(patch_types)):
    sig['%d'%(i+1)] = 2.0
    eps['%d'%(i+1)] = 0.5

rigid=hoomd.md.constrain.rigid()

##----------Patch Group 1----------##
coord_patch_group_1 = [[np.cos(Angle(60)),0,np.sin(Angle(60))],
                       [np.cos(Angle(60)),0,-np.sin(Angle(60))]]
mass_patch_group_1 = [0.5,0.5]
types_patch_group_1 = ['1','2']
diameters_patch_group_1 = [1.0,1.0]

##----------Patch Group 2----------##
coord_patch_group_2 = [[np.cos(Angle(60)),0,np.sin(Angle(60))],
                       [np.cos(Angle(60)),0,-np.sin(Angle(60))]]
mass_patch_group_2 = [0.5,0.5]
types_patch_group_2 = ['3','4']
diameters_patch_group_2 = [1.0,1.0]

##----------Calculates Moments of Inertia----------##
eigen_group_1 = MoI.Moment(coord_patch_group_1, mass_patch_group_1, debug=False)
eigen_group_2 = MoI.Moment(coord_patch_group_2, mass_patch_group_2, debug=False)

##----------Sets Rigid Parameters----------##
rigid.set_param('C0',positions=coord_patch_group_1, types=['1','2'], diameters = [1.0,1.0])
rigid.set_param('C9',positions=coord_patch_group_2, types=['3','4'], diameters = [1.0,1.0])

rigid.create_bodies()

##----------Creates groups----------##
    ##Grouping rigid centers after replication breaks everything.##
    ##Extra groups for debugging and/or flexability##
center = hoomd.group.rigid_center()
nonrigid = hoomd.group.nonrigid()
part = hoomd.group.all()
gpoly = hoomd.group.union(name='gpoly', a=center, b=nonrigid) 
patches = hoomd.group.difference(name = 'patches', a=part, b=gpoly) 

for p in gpoly:
    p.mass = 3.0
for p in patches:
    p.mass = 0.5
for p in center:
    p.moment_inertia = [0,0,0]

system.particles[0].moment_inertia = eigen_group_1
system.particles[9].moment_inertia = eigen_group_2

Poly.MakeBonds(nparticles, debug=False)

system.replicate(nx=5,ny=5,nz=5)

harm = md.bond.harmonic()
harm.bond_coeff.set('tether',k=250,r0=0.83)


nl = md.nlist.cell(r_buff = 0.95, check_period = 1)
#nl.tune(warmup=5e4, r_min=0.05, r_max=1.0, jumps=20, steps=5e4, r_buff)
#nl.tune(warmup=200000, r_min=0.05, r_max=1.0, jumps=20, steps=5000, set_max_check_period=False, quiet=False)
lj = md.pair.lj(r_cut=rcut, nlist=nl)
tmp1,tmp2 = 0.0,0.0

#Generates LJ pairs
for i in range(0,len(sig)):
    for j in range(i,len(sig)):
        lj.pair_coeff.set(types[i],types[j],
                epsilon=0.5*(eps[types[i]]+eps[types[j]]),
                sigma=0.5*(sig[types[i]]+sig[types[j]]))

        
#Integrators. 
hoomd.md.integrate.mode_standard(dt=0.002)
#hoomd.md.integrate.langevin(group=part, kT=0.1, seed = 5)
hoomd.md.integrate.langevin(group=nonrigid, kT=0.1, seed = SEED)
hoomd.md.integrate.langevin(group=center, kT=0.1, seed = SEED)



### DUMP AND RUN ###
hoomd.analyze.log(filename='random.log',quantities=['temperature','num_particles','ndof',
                                                    'translational_ndof','rotational_ndof',
                                                    'potential_energy'],period=1000,header_prefix='#'
                                                      ,overwrite=True)
#hoomd.deprecated.dump.xml(group=part,filename ='random.xml',vis=True,image=True)
hoomd.dump.gsd(filename='{0} {1}.gsd'.format(filename, t),period=1000, group = hoomd.group.all() ,overwrite = True, dynamic=['attribute'])
hoomd.run(1e5)

Debug Message Format Is As Follows:
 DEBUG: Function: Parameter = Value at line <linenumber>
notice(2): Group "all" created containing 10 particles
notice(2): constrain.rigid(): Creating 2 rigid bodies (adding 4 particles)
notice(2): Group "rigid_center" created containing 2 particles
notice(2): Group "nonrigid" created containing 8 particles
notice(2): Group "gpoly" created containing 10 particles
notice(2): Group "patches" created containing 4 particles
notice(2): integrate.langevin/bd is using specified gamma values
notice(2): integrate.langevin/bd is using specified gamma values
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 500
notice(2): Particles with 1 exclusions             : 250
notice(2): Particles with 2 exclusions             : 1000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **


*Warning*: Integrator #0: Anisotropic integration requested, but no rotational degrees of freedom found for its group


Time 00:00:10 | Step 15059 / 100000 | TPS 1505.84 | ETA 00:00:56
Time 00:00:20 | Step 28055 / 100000 | TPS 1299.51 | ETA 00:00:55
Time 00:00:30 | Step 40724 / 100000 | TPS 1266.88 | ETA 00:00:46
Time 00:00:40 | Step 52688 / 100000 | TPS 1196.39 | ETA 00:00:39
Time 00:00:50 | Step 64729 / 100000 | TPS 1204.09 | ETA 00:00:29
Time 00:01:00 | Step 76614 / 100000 | TPS 1188.42 | ETA 00:00:19
Time 00:01:10 | Step 88379 / 100000 | TPS 1176.44 | ETA 00:00:09
Time 00:01:20 | Step 99827 / 100000 | TPS 1144.74 | ETA 00:00:00
Time 00:01:20 | Step 100000 / 100000 | TPS 1148.85 | ETA 00:00:00
Average TPS: 1247.59
---------
-- Neighborlist stats:
0 normal updates / 1000 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 55 / n_neigh_avg: 18.9474
shortest rebuild period: 100
-- Cell list stats:
Dimension: 15, 5, 5
n_min    : 0 / n_max: 29 / n_avg: 4.66667
** run complete **
